In [73]:
# messy imports to be tidied up later
import os
import xlutils, xypath
import databaker
import databaker.constants
from databaker.constants import *
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, savepreviewhtmlBAGS, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [74]:
# input-output
inputfile = "annualfdi2014inwarddirectionalv3_tcm77-426424.xls"
outputfile = "%s.csv" % os.path.splitext(inputfile)[0]
previewfile = "preview.html"

#bake --preview FDI_Dataset7_recipe.py annualfdi2014inwarddirectionalv3_tcm77-426424.xls "Flow" "Inward"
#bake --preview FDI_Dataset7_recipe.py annualfdi2014outwarddirectionalv3_tcm77-426426.xls "Flow" "Outward"
#python FDI_Dataset_transform_7.py "data-annualfdi2014inwarddirectionalv3_tcm77-426424-FDI_Dataset7_recipe-Flow,Inward.csv" "data-annualfdi2014outwarddirectionalv3_tcm77-426426-FDI_Dataset7_recipe-Flow,Outward.csv" "Transform_Dataset7"


In [75]:
print("Loading %s which has size %d bytes" % (inputfile, os.path.getsize(inputfile)))
tableset = xypath.loader.table_set(inputfile, extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names: %s" % ", ".join([tab.name  for tab in tabs]))


Loading annualfdi2014inwarddirectionalv3_tcm77-426424.xls which has size 659456 bytes
Table names: Contents, 1.1, 1.2, 1.3, 2.1, 2.2, 2.3, 3.1, 3.2, 3.3, 4.1, 4.2, 4.3, Geography, SIC


In [76]:
# Normal place for your recipe work
conversionsegments = [ ]

def remove_below(bag, tab):
    """ Custom function. If the bag has two cells touching each other vertically get rid of the lower one"""
    
    previous_y = -2     # we're looking for vertical repeats, -2 is outside the 0 indexed axis so will do to start us off
    droplist = []       # for y property (think "row number" + 1) of the cells we're chucking


    # extrude to grab any 2 line area names
    for cell in bag:
        cell = cell.extrude(1,0)        

    # now get rid of and duplicates
    for cell in bag:
        if (cell.y - 1) == previous_y:
            droplist.append(cell.y + 1)            
        previous_y = cell.y
        
    for _y in droplist:
        bag = bag - tab.excel_ref('A' + str(_y)).expand(RIGHT)

    return bag
    
    
def waterfall(tab):
    # Get A
    colA = remove_below(tab.excel_ref('A').is_not_blank().is_not_whitespace(), tab)
    
    # Get B
    colB = remove_below(tab.excel_ref('B').is_not_blank().is_not_whitespace(), tab)
    colB = colB | colA.shift(RIGHT)
    colB = colB - colB.filter(contains_string("of which"))
    # Get C
    colC = remove_below(tab.excel_ref('C').is_not_blank().is_not_whitespace(), tab)
    colC = colC | colB.shift(RIGHT)    
    
    dimensions = [ (colA, "AreaA", CLOSEST, ABOVE),
                   (colB, "AreaB", CLOSEST, ABOVE), 
                   (colC, "AreaC", CLOSEST, ABOVE)    
                 ]
    
    return dimensions



for tab in tabs:
    if tab.name not in ["2.3", "3.3", "4.3"]:
        continue
    anchor = tab.filter(contains_string (' fishing')).assert_one()
    
    obs = anchor.fill(DOWN).expand(RIGHT).is_not_blank()
    unwanted = tab.filter(contains_string ('Source: Office'))
    obs = obs-unwanted

    
    dimensions = [ 
        (anchor.shift(LEFT).expand(DOWN).is_not_blank(), TIME, DIRECTLY, LEFT),
        (anchor.expand(RIGHT).parent().is_not_blank(), "Investment", DIRECTLY, ABOVE),
        ("Inward", "Flow", None, None), 
        (tab.excel_ref('A2'), 'Type', CLOSEST, ABOVE)    
    ]
    
    dimensions.extend(waterfall(tab))
    conversionsegments.append((tab, dimensions, obs))





In [77]:
    
# this is the preview system
conversionsegment = conversionsegments[-1]
savepreviewhtml(conversionsegment, batchcelllookup, previewfile)


opening file preview.html
table '4.3' written
javascript calculated


In [15]:
print("converting and writing %d conversion segments into %s" % (len(conversionsegments), outputfile))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment, batchcelllookup)
    convertedrows.append(rows)
    
# Should detect errors and throw up preview with cells marked
# (though actually the errors manifest further upstream, normally)
    
# here we can do further sorting and postprocessing and allocating to different files 

writetechnicalCSV(outputfile, conversionsegments, convertedrows)



converting and writing 3 conversion segments into annualfdi2014inwarddirectionalv3_tcm77-426424.csv
conversion segment size 4256
conversion segment size 4255
conversion segment size 4256
converting and writing 3 conversion segments into annualfdi2014inwarddirectionalv3_tcm77-426424.csv
conversion segment size 4256
conversion segment size 4255
conversion segment size 4256


In [40]:
1


1

In [69]:
import pandas as pd
import numpy as np
df = pd.DataFrame.from_dict(convertedrows[0])
#df[df["AreaC"] == "DENMARK" and df[OBS].map(np.isreal)]
k = df[df[OBS].map(np.isreal)][df["AreaC"] == "DENMARK"]
#df[df[OBS].isnumeric()]
#df[OBS].isnumeric()


/home/goatchurch/sensiblecode/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [72]:
k.sort(OBS)

/home/goatchurch/sensiblecode/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,AreaA,AreaB,AreaC,Investment,Type,-9,Flow,-2
664,EUROPE,EU,DENMARK,Total,2.3 Foreign direct investment flows into the U...,-1291,Inward,2013.0
622,EUROPE,EU,DENMARK,Financial services,2.3 Foreign direct investment flows into the U...,-108,Inward,2011.0
642,EUROPE,EU,DENMARK,"Professional, scientific & technical services",2.3 Foreign direct investment flows into the U...,-60,Inward,2012.0
640,EUROPE,EU,DENMARK,Information and communication,2.3 Foreign direct investment flows into the U...,-30,Inward,2012.0
635,EUROPE,EU,DENMARK,Other manufacturing,2.3 Foreign direct investment flows into the U...,-7,Inward,2012.0
616,EUROPE,EU,DENMARK,Other manufacturing,2.3 Foreign direct investment flows into the U...,-3,Inward,2011.0
611,EUROPE,EU,DENMARK,Textiles & wood activities,2.3 Foreign direct investment flows into the U...,0,Inward,2011.0
634,EUROPE,EU,DENMARK,Transport equipment,2.3 Foreign direct investment flows into the U...,0,Inward,2012.0
633,EUROPE,EU,DENMARK,"Computer, electronic and optical products",2.3 Foreign direct investment flows into the U...,1,Inward,2012.0
613,EUROPE,EU,DENMARK,Metal and machinery products,2.3 Foreign direct investment flows into the U...,2,Inward,2011.0
